In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import metrics
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy

plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [31]:
data1 = pd.read_excel('used_car_train data2.xls', encoding='gbk').set_index(["SaleID", "name"])
data1

,,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,name,,,,,,,,,,,,,,,,,,,,,
0,736,20040402,30,6,1.0,0.0,0.0,60,12.5,0,1046,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,2262,20030301,40,1,2.0,0.0,0.0,0,15.0,-,4366,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,14874,20040403,115,15,1.0,0.0,0.0,163,12.5,0,2806,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,71865,19960908,109,10,0.0,0.0,1.0,193,15.0,0,434,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,111080,20120103,110,5,1.0,0.0,0.0,68,5.0,0,6977,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,189536,20071110,1,1,2.0,NaN,0.0,140,15.0,0,1343,...,0.261753,0.000352,0.138976,0.085219,0.025476,2.106361,-2.359151,2.165224,-0.846520,1.306796
2997,63470,20020204,145,4,5.0,0.0,1.0,343,15.0,0,5751,...,0.283772,0.134105,0.104052,0.061334,0.002554,-7.161608,0.333974,2.171960,-2.240909,-1.597441
2998,5626,20000607,26,14,2.0,0.0,0.0,101,15.0,0,1463,...,0.253778,0.090059,0.056603,0.024048,0.069122,-2.377253,1.792449,-2.847012,0.154435,0.118546


In [32]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3001 entries, (0, 736) to (3000, 59888)
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   regDate            3001 non-null   int64  
 1   model              3001 non-null   int64  
 2   brand              3001 non-null   int64  
 3   bodyType           2922 non-null   float64
 4   fuelType           2827 non-null   float64
 5   gearbox            2876 non-null   float64
 6   power              3001 non-null   int64  
 7   kilometer          3001 non-null   float64
 8   notRepairedDamage  3001 non-null   object 
 9   regionCode         3001 non-null   int64  
 10  seller             3001 non-null   int64  
 11  offerType          3001 non-null   int64  
 12  creatDate          3001 non-null   int64  
 13  price              3001 non-null   int64  
 14  v_0                3001 non-null   float64
 15  v_1                3001 non-null   float64
 16  v_2     

In [33]:
data1.dropna(axis=0,how='all')
data1 = data1[~data1['bodyType'].isin(["NAN"])]
data1 = data1[~data1['fuelType'].isin(["NAN"])]
data1 = data1[~data1['gearbox'].isin(["NAN"])]
data1 = data1[~data1['notRepairedDamage'].isin(["-"])]
data1 = data1.astype('float')
data1

,,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,name,,,,,,,,,,,,,,,,,,,,,
0,736,20040402.0,30.0,6.0,1.0,0.0,0.0,60.0,12.5,0.0,1046.0,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
2,14874,20040403.0,115.0,15.0,1.0,0.0,0.0,163.0,12.5,0.0,2806.0,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,71865,19960908.0,109.0,10.0,0.0,0.0,1.0,193.0,15.0,0.0,434.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,111080,20120103.0,110.0,5.0,1.0,0.0,0.0,68.0,5.0,0.0,6977.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
5,137642,20090602.0,24.0,10.0,0.0,1.0,0.0,109.0,10.0,0.0,3690.0,...,0.260246,0.000518,0.119838,0.090922,0.048769,1.885526,-2.721943,2.457660,-0.286973,0.206573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1674,19991108.0,0.0,0.0,0.0,0.0,0.0,150.0,15.0,0.0,1423.0,...,0.267034,0.108669,0.084197,0.034922,0.031093,-4.240902,1.423684,-1.069548,-1.080304,0.330602
2997,63470,20020204.0,145.0,4.0,5.0,0.0,1.0,343.0,15.0,0.0,5751.0,...,0.283772,0.134105,0.104052,0.061334,0.002554,-7.161608,0.333974,2.171960,-2.240909,-1.597441
2998,5626,20000607.0,26.0,14.0,2.0,0.0,0.0,101.0,15.0,0.0,1463.0,...,0.253778,0.090059,0.056603,0.024048,0.069122,-2.377253,1.792449,-2.847012,0.154435,0.118546


In [34]:
#zscore标准化
values = data1.values            #Dataframe转换为Array
values = values.astype('float32')       #定义数据类型
scaler = preprocessing.StandardScaler() #创建Scaler对象
temp = scaler.fit_transform(values)     #稀疏矩阵  
z_score = pd.DataFrame(temp,columns=data1.columns,index = data1.index) #将Array还原为Dataframe，保持行列名
z_score = z_score.drop(["seller","offerType", "regDate"], axis=1) # 这两行毫无变化，无需使用，regDate 注册时间无用

In [35]:
y = z_score.iloc[:,13]
x = z_score.drop(["price"], axis=1)

In [36]:
# 数据分割 训练集 和 测试集
kf = KFold(n_splits = 10)
dict_datas = {}
count = 0
for train,valid in kf.split(x):
    train_x = np.array(x)[train]
    valid_x = np.array(x)[valid]
    train_x = pd.DataFrame(train_x,columns=list(z_score.columns.delete(list(z_score.columns).index("price"))))
    valid_x = pd.DataFrame(valid_x,columns=list(z_score.columns.delete(list(z_score.columns).index("price"))))
    train_y = np.array(y)[train]
    valid_y = np.array(y)[valid]
    train_y = pd.DataFrame(train_y,columns=["price"])
    valid_y = pd.DataFrame(valid_y,columns=["price"])
    dict_datas[count] = (train_x, valid_x,train_y, valid_y)
    count += 1

In [37]:
x_model = sm.add_constant(x)
model = sm.OLS(y,x_model)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    v_2   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.270e+21
Date:                Wed, 29 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:08:52   Log-Likelihood:                 48377.
No. Observations:                2347   AIC:                        -9.670e+04
Df Residuals:                    2321   BIC:                        -9.655e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1.812e-16    6.2e-12   2.92e-05      1.000   -1.22e-11    1.22e-11
model             -1.049e-15    7.9e-12     -0.000      1.000   -1.55e-11    1.55e-11
brand              2.204e-16   6.53e-12   3.38e-05      1.000   -1.28e-11    1.28e-11
bodyType           9.226e-16   7.21e-12      0.000      1.000   -1.41e-11    1.41e-11
fuelType          -6.863e-17    6.4e-12  -1.07e-05      1.000   -1.26e-11    1.26e-11
gearbox           -6.614e-16   6.36e-12     -0.000      1.000   -1.25e-11    1.25e-11
power              1.033e-15   7.48e-12      0.000      1.000   -1.47e-11    1.47e-11
kilometer          8.342e-16   7.68e-12      0.000      1.000   -1.51e-11    1.51e-11
notRepairedDamage -2.212e-17   5.99e-12   -3.7e-06      1.000   -1.17e-11    1.17e-11
regionCode         2.194e-16   6.07e-12   3.61e-05      1.000   -1.19e-11    1.19e-11
creatDate         -2.262e-16   5.66e-12     -4e-05      1.000   -1.11e-11    1.11e-11
v_0               -1.495e-11   2.42e-06  -6.18e-06      1.000   -4.75e-06    4.75e-06
v_1               -3.247e-10      0.000  -2.93e-06      1.000      -0.000       0.000
v_2                   1.0000   9.42e-06   1.06e+05      0.000       1.000       1.000
v_3               -1.165e-12   1.25e-06  -9.32e-07      1.000   -2.45e-06    2.45e-06
v_4               -5.862e-14   1.49e-08  -3.93e-06      1.000   -2.92e-08    2.92e-08
v_5               -1.066e-14   1.26e-08  -8.45e-07      1.000   -2.47e-08    2.47e-08
v_6               -1.721e-14   3.81e-09  -4.52e-06      1.000   -7.46e-09    7.46e-09
v_7               -6.695e-14   6.68e-09     -1e-05      1.000   -1.31e-08    1.31e-08
v_8                -8.46e-14   6.98e-09  -1.21e-05      1.000   -1.37e-08    1.37e-08
v_9               -7.905e-14   8.05e-09  -9.82e-06      1.000   -1.58e-08    1.58e-08
v_10              -4.247e-10   8.71e-05  -4.87e-06      1.000      -0.000       0.000
v_11                2.16e-10   3.73e-05   5.79e-06      1.000   -7.31e-05    7.31e-05
v_12               4.593e-11   9.27e-06   4.95e-06      1.000   -1.82e-05    1.82e-05
v_13              -6.439e-14    3.3e-08  -1.95e-06      1.000   -6.47e-08    6.47e-08
v_14               2.214e-15   5.16e-10   4.29e-06      1.000   -1.01e-09    1.01e-09
==============================================================================
Omnibus:                    11569.191   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              275.352
Skew:                          -0.191   Prob(JB):                     1.61e-60
Kurtosis:                       1.366   Cond. No.                     6.28e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smalles

In [38]:
#拟合优度
results.rsquared_adj

1.0

In [39]:
#mses = []
maes = []
xs = []
results_list = []
for i in dict_datas:
    
    # 筛斯皮尔曼相关系数大于0.几的变量
    x = dict_datas[i][0].copy()
    y = dict_datas[i][2].copy()
    x1 = x.copy()
    x1["price"]=y
    sp = abs(x1.corr('spearman'))
    sp.sort_values(by="price" , inplace=True, ascending=False)
    temp1 = []
    for j in sp["price"][2:]: 
        # 上面从位置3开始是因为要去除相关度最高的两个，一个是y本身，一个是var_1,它相关度为1意味着它就是y的另一种表现形式，需要去除，我们需要筛x的
        if j >= 0.1: # 相关度限额，控制最终几个变量
            temp1.append((sp["price"][1:].index)[list(sp["price"][1:]).index(j)])
    x = x[temp1]
    
    # 正式回归
    x_model = sm.add_constant(x)
    model = sm.OLS(y,x_model)
    results = model.fit()
    results_list.append(results)
    xs.append(results.params)
    
    # 测试集检验
    temps = []
    for j in range(len(dict_datas[i][1][temp1])):
        temp = np.dot(np.array(results.params[1:]),np.array(dict_datas[i][1][temp1])[j])+np.array(results.params)[0]
        temps.append(temp)
    temps = np.array(temps)
    MAE = np.mean(np.abs(np.array(dict_datas[i][3].T)[0]-temps))
    maes.append(MAE)

In [40]:
#预测集最小maes
min(maes)

0.00031783686167601765

In [41]:
#预测集最小maes所在的k折检验中的最后一折里
maes.index(min(maes))

9

In [42]:
#对应报告
results_list[maes.index(min(maes))].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.764e+08
Date:                Wed, 29 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:08:55   Log-Likelihood:                 12795.
No. Observations:                2113   AIC:                        -2.555e+04
Df Residuals:                    2095   BIC:                        -2.545e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -2.752e-06   1.24e-05     -0.222      0.824   -2.71e-05    2.16e-05
v_12                  2.3565      0.002   1436.267      0.000       2.353       2.360
v_0                   1.1637      0.002    538.692      0.000       1.159       1.168
v_7                   0.0011      0.000      8.609      0.000       0.001       0.001
v_3                   0.5927      0.002    362.100      0.000       0.590       0.596
v_8                   0.0059      0.001      7.148      0.000       0.004       0.007
v_6                   0.0091      0.004      2.174      0.030       0.001       0.017
v_10                  9.4133      0.008   1223.882      0.000       9.398       9.428
kilometer          3.227e-05    1.7e-05      1.895      0.058   -1.13e-06    6.57e-05
power             -2.341e-05   1.62e-05     -1.448      0.148   -5.51e-05     8.3e-06
v_14                  0.0007    3.5e-05     19.714      0.000       0.001       0.001
v_1                   9.3659      0.009   1100.955      0.000       9.349       9.383
gearbox            1.271e-05    1.4e-05      0.908      0.364   -1.47e-05    4.02e-05
fuelType            5.35e-05   1.41e-05      3.788      0.000    2.58e-05    8.12e-05
notRepairedDamage -2.657e-05   1.33e-05     -1.993      0.046   -5.27e-05   -4.28e-07
brand             -7.465e-05   1.41e-05     -5.313      0.000      -0.000   -4.71e-05
v_9                   0.0048      0.000     40.458      0.000       0.005       0.005
bodyType          -3.047e-07   1.59e-05     -0.019      0.985   -3.15e-05    3.09e-05
==============================================================================
Omnibus:                     2061.673   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           236583.991
Skew:                          -4.320   Prob(JB):                         0.00
Kurtosis:                      54.113   Cond. No.                     2.00e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#训练集拟合优度
results_list[maes.index(min(maes))].rsquared_adj

In [58]:
# 参数
results_list[maes.index(min(maes))].params

const               -2.751783e-06
v_12                 2.356544e+00
v_0                  1.163682e+00
v_7                  1.107320e-03
v_3                  5.927439e-01
v_8                  5.863745e-03
v_6                  9.055166e-03
v_10                 9.413295e+00
kilometer            3.227047e-05
power               -2.340975e-05
v_14                 6.902462e-04
v_1                  9.365865e+00
gearbox              1.270985e-05
fuelType             5.349959e-05
notRepairedDamage   -2.656714e-05
brand               -7.465249e-05
v_9                  4.823494e-03
bodyType            -3.046784e-07
dtype: float64

In [111]:
#测试集最后一行的模型的预测值
ori1 = dict_datas[9][1][list(results_list[maes.index(min(maes))].params.index)[1:]]
temp = 0
count = 0
for i,j in ori1.iteritems():
    k = results_list[maes.index(min(maes))].params[1:][count]
    temp+=(j*k)[233]
temp+results_list[maes.index(min(maes))].params[0]

2.0028689403975575

In [110]:
#测试集最后一行的实际值
dict_datas[9][-1].values[-1][0]

2.1106617